<a href="https://colab.research.google.com/github/romitbarua/MultiModalDeepFake/blob/main/UtilityNotebooks/GenerateMetadataFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
#mount the google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [68]:
import os
import pandas as pd
import cv2
import dlib

In [69]:
SYM_PATH = '/content/drive/MyDrive/DeepFakeDetection'
%cd $SYM_PATH
%pip install -e .

/content/drive/MyDrive/DeepFakeDetection
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/DeepFakeDetection
  Attempting uninstall: DeepFake
    Found existing installation: DeepFake 0.1.0
    Can't uninstall 'DeepFake'. No files were found to uninstall.
  Running setup.py develop for DeepFake


In [70]:
from packages.DlibManager import DlibManager

In [71]:
#assign major directories
data_root_directory = '/content/drive/MyDrive/DeepFakeDetection/data'
VoxCeleb_directory = '/content/drive/MyDrive/DeepFakeDetection/data/VoxCeleb'
phoneme_directory = 'MfaPhonemeAlignment'
utterance_directory = 'WhisperUtterances'

curr_metadata = pd.read_csv('/content/drive/MyDrive/DeepFakeDetection/data/full_metadata.csv')

In [114]:
#settings
regenerate_FakeAVCeleb_df = False
regenerate_VoxCeleb_df = True

check_wav_FakeAVCeleb = True
check_wav_VoxCeleb = True

check_utterance_FakeAVCeleb = True
check_utterance_VoxCeleb = True

check_phoneme_FakeAVCeleb = True
check_phoneme_VoxCeleb = True

check_dlib_validity_FakeAVCeleb = False
check_dlib_validity_VoxCeleb = True



In [73]:
#utility function
def dlib_validity(video_paths):

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('/content/drive/MyDrive/DeepFakeDetection/model/shape_predictor_68_face_landmarks.dat')

    dlib_valid = []
    for idx, video_path in enumerate(video_paths):

        print(f'\r Video #{idx+1} out of {len(video_paths)}', end='')
        video = cv2.VideoCapture(video_path)

        try:
            dlib_video = DlibManager(detector=detector, predictor=predictor, video=video)
            dlib_valid.append(True)
        except:
            dlib_valid.append(False)

    return dlib_valid



In [74]:
if regenerate_FakeAVCeleb_df:
    fake_av_celeb_metadata = pd.read_csv('/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/meta_data.csv')
    fake_av_celeb_metadata.rename(columns={'Unnamed: 9':'directory'}, inplace=True)

    fake_av_celeb_metadata['directory'] = fake_av_celeb_metadata['directory'].str.replace('FakeAVCeleb', 'FakeAVCeleb_v1.2')
    fake_av_celeb_metadata['full_path'] = data_root_directory + '/' +fake_av_celeb_metadata['directory'] + '/' + fake_av_celeb_metadata['path'] 
    fake_av_celeb_metadata['video_source'] = 'FakeAVCeleb'
else:
    fake_av_celeb_metadata = curr_metadata[curr_metadata['video_source'] == 'FakeAVCeleb'] 

In [75]:
fake_av_celeb_metadata.head()

,Unnamed: 0,source,path,directory,full_path,wav_path,valid_dlib,type,method,video_source,target1,target2,category,race,gender,phoneme_path
1394,0,id00076,00109.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,True,RealVideo-RealAudio,real,FakeAVCeleb,-,-,A,African,men,/content/drive/MyDrive/DeepFakeDetection/data/...
1395,1,id00166,00010.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,True,RealVideo-RealAudio,real,FakeAVCeleb,-,-,A,African,men,/content/drive/MyDrive/DeepFakeDetection/data/...
1396,2,id00173,00118.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,True,RealVideo-RealAudio,real,FakeAVCeleb,-,-,A,African,men,/content/drive/MyDrive/DeepFakeDetection/data/...
1397,3,id00366,00118.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,False,RealVideo-RealAudio,real,FakeAVCeleb,-,-,A,African,men,/content/drive/MyDrive/DeepFakeDetection/data/...
1398,4,id00391,00052.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,True,RealVideo-RealAudio,real,FakeAVCeleb,-,-,A,African,men,/content/drive/MyDrive/DeepFakeDetection/data/...


In [91]:
#get the wavfiles, utterances & phonemes

phoneme_paths = []
wav_paths = []
utterance_paths = []

for index, row in fake_av_celeb_metadata.iterrows():

    print(f'\rVideo #{index+1} out of {fake_av_celeb_metadata.shape[0]}', end='')

    if check_wav_FakeAVCeleb:
        wav_path = row['full_path'].replace('.mp4', '.wav')
        if os.path.exists(wav_path):
            wav_paths.append(wav_path)
        else:
            wav_paths.append('')
    else:
        wav_paths.append(row['wav_path'])
    
    if check_utterance_FakeAVCeleb:
        #utterance_path = os.path.join(data_root_directory, utterance_directory, row['directory'], row['path']).replace('.mp4', '.txt')
        utterance_path = f'{data_root_directory}/{utterance_directory}/{row["directory"]}/{ row["path"]}'.replace('.mp4', '.txt')
        
        if os.path.exists(utterance_path):
            print('found utterance')
            utterance_paths.append(utterance_path)
        else:
            utterance_paths.append('')
    else:
        utterance_paths.append(row['utterance_path'])


    if check_phoneme_FakeAVCeleb:
        phoneme_path = os.path.join(data_root_directory, phoneme_directory, row['directory'], row['path']).replace('.mp4', '.TextGrid')
        #phoneme_path = row['full_path'].replace('FakeAVCeleb_v1.2', fake_av_phoneme_directory).replace('.mp4', '.TextGrid')
        if os.path.exists(phoneme_path):
            phoneme_paths.append(phoneme_path)
        else:
            phoneme_paths.append('')
    else:
        phoneme_paths.append(row['phoneme_path'])


    
fake_av_celeb_metadata['wav_path'] = wav_paths
fake_av_celeb_metadata['utterance_path'] = utterance_paths
fake_av_celeb_metadata['phoneme_path'] = phoneme_paths

Video #1395 out of 21566found utterance
Video #1396 out of 21566found utterance
Video #1397 out of 21566found utterance
Video #1398 out of 21566found utterance
Video #1399 out of 21566found utterance
Video #1400 out of 21566found utterance
Video #1401 out of 21566found utterance
Video #1402 out of 21566found utterance
Video #1403 out of 21566found utterance
Video #1404 out of 21566found utterance
Video #1405 out of 21566found utterance
Video #1406 out of 21566found utterance
Video #1407 out of 21566found utterance
Video #1408 out of 21566found utterance
Video #1409 out of 21566found utterance
Video #1410 out of 21566found utterance
Video #1411 out of 21566found utterance
Video #1412 out of 21566found utterance
Video #1413 out of 21566found utterance
Video #1414 out of 21566found utterance
Video #1415 out of 21566found utterance
Video #1416 out of 21566found utterance
Video #1417 out of 21566found utterance
Video #1418 out of 21566found utterance
Video #1419 out of 21566found utterance


<ipython-input-91-dd8b5e4698f9>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_av_celeb_metadata['wav_path'] = wav_paths
<ipython-input-91-dd8b5e4698f9>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_av_celeb_metadata['utterance_path'] = utterance_paths
<ipython-input-91-dd8b5e4698f9>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [77]:
fake_av_celeb_metadata.head()

,Unnamed: 0,source,path,directory,full_path,wav_path,valid_dlib,type,method,video_source,target1,target2,category,race,gender,phoneme_path,utterance_path
1394,0,id00076,00109.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,True,RealVideo-RealAudio,real,FakeAVCeleb,-,-,A,African,men,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...
1395,1,id00166,00010.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,True,RealVideo-RealAudio,real,FakeAVCeleb,-,-,A,African,men,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...
1396,2,id00173,00118.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,True,RealVideo-RealAudio,real,FakeAVCeleb,-,-,A,African,men,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...
1397,3,id00366,00118.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,False,RealVideo-RealAudio,real,FakeAVCeleb,-,-,A,African,men,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...
1398,4,id00391,00052.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,True,RealVideo-RealAudio,real,FakeAVCeleb,-,-,A,African,men,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...


In [78]:
if check_dlib_validity_FakeAVCeleb:
    fake_av_celeb_metadata['valid_dlib'] = dlib_validity(list(fake_av_celeb_metadata['full_path']))

In [108]:
if regenerate_VoxCeleb_df:
    print('Here')
    ids = list(os.walk(VoxCeleb_directory))[0][1]

    sources = []
    paths = []
    directories = []
    full_paths = []

    for id in ids:
        
        id_directory = f'{VoxCeleb_directory}/{id}'
        for walk_path in os.walk(id_directory):
            files = walk_path[2]
            for file_name in files:
                if '.mp4' in file_name:
                    sources.append(id)
                    paths.append(file_name)
                    directories.append(id_directory.replace('/content/drive/MyDrive/DeepFakeDetection/data/', ''))

                    full_path = os.path.join(id_directory, file_name)
                    full_paths.append(full_path)

    
    vox_celeb_metadata = pd.DataFrame({'source':sources, 'path':paths, 'directory':directories, 'full_path':full_paths})
    
    vox_celeb_metadata['type'] = 'RealVideo-RealAudio'
    vox_celeb_metadata['method'] = 'real'
    vox_celeb_metadata['video_source'] = 'VoxCeleb'

else:
    vox_celeb_metadata = curr_metadata[curr_metadata['video_source'] == 'VoxCeleb'] 

#Generate VoxCeleb Metadata file


Here
['id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id01163', 'id0

In [110]:
#get the wavfiles, utterances & phonemes

phoneme_paths = []
wav_paths = []
utterance_paths = []

for index, row in vox_celeb_metadata.iterrows():

    print(f'\rVideo #{index+1} out of {vox_celeb_metadata.shape[0]}', end='')

    if check_wav_VoxCeleb:
        wav_path = row['full_path'].replace('.mp4', '.wav')
        if os.path.exists(wav_path):
            wav_paths.append(wav_path)
        else:
            wav_paths.append('')
    else:
        wav_paths.append(row['wav_path'])
    
    if check_utterance_VoxCeleb:
        #utterance_path = os.path.join(data_root_directory, utterance_directory, row['directory'], row['path']).replace('.mp4', '.txt')
        print(utterance_directory)
        print(utterance_path)
        utterance_path = f'{data_root_directory}/{utterance_directory}/{row["directory"]}/{ row["path"]})'.replace('.mp4', '.txt')
        if os.path.exists(utterance_path):
            utterance_paths.append(utterance_path)
        else:
            utterance_paths.append('')
    else:
        utterance_paths.append(row['utterance_path'])


    if check_phoneme_VoxCeleb:
        phoneme_path = os.path.join(data_root_directory, phoneme_directory, row['directory'], row['path']).replace('.mp4', '.TextGrid')
        #phoneme_path = row['full_path'].replace('FakeAVCeleb_v1.2', fake_av_phoneme_directory).replace('.mp4', '.TextGrid')
        if os.path.exists(phoneme_path):
            phoneme_paths.append(phoneme_path)
        else:
            phoneme_paths.append('')
    else:
        phoneme_paths.append(row['phoneme_path'])


    
vox_celeb_metadata['wav_path'] = wav_paths
vox_celeb_metadata['utterance_path'] = utterance_paths
vox_celeb_metadata['phoneme_path'] = phoneme_paths

Video #1 out of 1394WhisperUtterances
/content/drive/MyDrive/DeepFakeDetection/data/WhisperUtterances/FakeAVCeleb_v1.2/FakeVideo-FakeAudio/Asian (South)/women/id07689/00028_id07686_wavtolip.txt
Video #2 out of 1394WhisperUtterances
/content/drive/MyDrive/DeepFakeDetection/data/WhisperUtterances/VoxCeleb/id01163/00500.txt)
Video #3 out of 1394WhisperUtterances
/content/drive/MyDrive/DeepFakeDetection/data/WhisperUtterances/VoxCeleb/id01163/00497.txt)
Video #4 out of 1394WhisperUtterances
/content/drive/MyDrive/DeepFakeDetection/data/WhisperUtterances/VoxCeleb/id01163/00493.txt)
Video #5 out of 1394WhisperUtterances
/content/drive/MyDrive/DeepFakeDetection/data/WhisperUtterances/VoxCeleb/id01163/00490.txt)
Video #6 out of 1394WhisperUtterances
/content/drive/MyDrive/DeepFakeDetection/data/WhisperUtterances/VoxCeleb/id01163/00496.txt)
Video #7 out of 1394WhisperUtterances
/content/drive/MyDrive/DeepFakeDetection/data/WhisperUtterances/VoxCeleb/id01163/00488.txt)
Video #8 out of 1394Whispe

In [115]:
if check_dlib_validity_VoxCeleb:
  vox_celeb_metadata['valid_dlib'] = dlib_validity(list(vox_celeb_metadata['full_path']))



 Video #1394 out of 1394

In [117]:
full_data = pd.concat((vox_celeb_metadata, fake_av_celeb_metadata))
full_data.columns

Index(['source', 'path', 'directory', 'full_path', 'type', 'method',
       'video_source', 'wav_path', 'utterance_path', 'phoneme_path',
       'valid_dlib', 'Unnamed: 0', 'target1', 'target2', 'category', 'race',
       'gender'],
      dtype='object')

In [118]:

full_data

,source,path,directory,full_path,type,method,video_source,wav_path,utterance_path,phoneme_path,valid_dlib,Unnamed: 0,target1,target2,category,race,gender
0,id01163,00500.mp4,VoxCeleb/id01163,/content/drive/MyDrive/DeepFakeDetection/data/...,RealVideo-RealAudio,real,VoxCeleb,,,,False,NaN,NaN,NaN,NaN,NaN,NaN
1,id01163,00497.mp4,VoxCeleb/id01163,/content/drive/MyDrive/DeepFakeDetection/data/...,RealVideo-RealAudio,real,VoxCeleb,,,,False,NaN,NaN,NaN,NaN,NaN,NaN
2,id01163,00493.mp4,VoxCeleb/id01163,/content/drive/MyDrive/DeepFakeDetection/data/...,RealVideo-RealAudio,real,VoxCeleb,/content/drive/MyDrive/DeepFakeDetection/data/...,,/content/drive/MyDrive/DeepFakeDetection/data/...,True,NaN,NaN,NaN,NaN,NaN,NaN
3,id01163,00490.mp4,VoxCeleb/id01163,/content/drive/MyDrive/DeepFakeDetection/data/...,RealVideo-RealAudio,real,VoxCeleb,/content/drive/MyDrive/DeepFakeDetection/data/...,,/content/drive/MyDrive/DeepFakeDetection/data/...,True,NaN,NaN,NaN,NaN,NaN,NaN
4,id01163,00496.mp4,VoxCeleb/id01163,/content/drive/MyDrive/DeepFakeDetection/data/...,RealVideo-RealAudio,real,VoxCeleb,,,,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22955,id07689,00028_id06254_wavtolip.mp4,FakeAVCeleb_v1.2/FakeVideo-FakeAudio/Asian (So...,/content/drive/MyDrive/DeepFakeDetection/data/...,FakeVideo-FakeAudio,wav2lip,FakeAVCeleb,/content/drive/MyDrive/DeepFakeDetection/data/...,,/content/drive/MyDrive/DeepFakeDetection/data/...,True,21561.0,id06254,-,D,Asian (South),women
22956,id07689,00028_id06343_wavtolip.mp4,FakeAVCeleb_v1.2/FakeVideo-FakeAudio/Asian (So...,/content/drive/MyDrive/DeepFakeDetection/data/...,FakeVideo-FakeAudio,wav2lip,FakeAVCeleb,/content/drive/MyDrive/DeepFakeDetection/data/...,,/content/drive/MyDrive/DeepFakeDetection/data/...,True,21562.0,id06343,-,D,Asian (South),women
22957,id07689,00028_id07008_wavtolip.mp4,FakeAVCeleb_v1.2/FakeVideo-FakeAudio/Asian (So...,/content/drive/MyDrive/DeepFakeDetection/data/...,FakeVideo-FakeAudio,wav2lip,FakeAVCeleb,/content/drive/MyDrive/DeepFakeDetection/data/...,,/content/drive/MyDrive/DeepFakeDetection/data/...,True,21563.0,id07008,-,D,Asian (South),women
22958,id07689,00028_id07377_wavtolip.mp4,FakeAVCeleb_v1.2/FakeVideo-FakeAudio/Asian (So...,/content/drive/MyDrive/DeepFakeDetection/data/...,FakeVideo-FakeAudio,wav2lip,FakeAVCeleb,/content/drive/MyDrive/DeepFakeDetection/data/...,,/content/drive/MyDrive/DeepFakeDetection/data/...,True,21564.0,id07377,-,D,Asian (South),women


In [119]:
full_data.to_csv('/content/drive/MyDrive/DeepFakeDetection/data/full_metadata.csv')